Make an Assistant which can provide the ecommerce product details if someone ask for a product.
1. Use Pydantic technics for output string format
2. Any LLM
3. Output format will be a. Product Name, b. Product details, c. Tentative Price in $

In [1]:
from pydantic import BaseModel,Field
from typing import Dict,Any
from langchain_core.output_parsers import JsonOutputParser
import requests
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

In [2]:
class product_item_details(BaseModel):
    product_name: str=Field(description="Product Name")
    product_details: Dict[str, Any]=Field(description="Product details")
    tentative_price: str=Field(description="This is tentative price of the product in $")
    rating: str=Field(description="Rating of the product out of 5")

In [3]:


details_output_parser=JsonOutputParser(pydantic_object=product_item_details)

In [4]:
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [20]:
details_prompt=ChatPromptTemplate.from_messages([
    ("system", """You are a helpful shopping assistant that answers questions using data provided by a custom function 
     when available. If the function returns relevant data, use it to form a clear, complete, and human-like response. 
     If the function output is missing or incomplete or No product found then use your own reasoning to provide the best possible answer. Product
     details will be in dictionary format. If multiple products are available for in custom function call provide the
     best rating and most sell and best price product details.
     Please exclude the price section from product details part.
     Product details will be like dimensions, ram size, disk size,product url etc its fully depends on product. If custom functions
     output do not have that much informations then check your own data and provide best and complete details information.
    ## Function Output:
    {function_output}
    ## User Question:
    {user_input}
    ## Your Answer:
    {format_instruction}"""),
    ("human","{user_input}")

]
)

In [6]:
details_chain=details_prompt.partial(format_instruction=details_output_parser.get_format_instructions()) | model | details_output_parser

In [7]:
def real_time_product(product_name):
    url = "https://real-time-amazon-data.p.rapidapi.com/search"
    querystring = {"query":product_name,"country":"US"}

    headers = {
	    "x-rapidapi-key": "8d2c3fac9emshdb7e04ae201905ep12a1e0jsn4d4b4f089f6c",
	    "x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
        }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()

In [8]:
extract_prompt=PromptTemplate(
    template="""
        You are an assistant that extracts product names from user questions.
        Extract the exact name of the product mentioned in the following {query}. Respond with only the product name and nothing else. If there is no product name in the query provide response as None""",
        input_variables="{query}"     
)

In [9]:
str_output_parser=StrOutputParser()

In [10]:
extract_chain=extract_prompt | model | str_output_parser



In [11]:
def product_details_output(product_name):
    if product_name=='None':
        return 'No product found'
    else:
        product_details=real_time_product(product_name)
        return product_details

In [12]:
def fetch_product_details(user_text):
    product_name=extract_chain.invoke(user_text)
    print(product_name)
    result=details_chain.invoke({"function_output":product_details_output(product_name),"user_input":product_name})
    return result

In [ ]:
# Real time product details from Amazon
result=fetch_product_details('I want to buy a samsung tv. can you suggest best price') 
print(result)

samsung tv
{'product_name': 'Samsung 50-Inch Class QLED Q60C Series Quantum HDR Smart TV', 'product_details': {'Resolution': '4K', 'Features': 'Quantum HDR, Dual LED, Object Tracking Sound Lite, Q-Symphony, Motion Xcelerator, Gaming Hub, Alexa Built-in', 'Model': 'QN50Q60C', 'Year': '2023', 'Product URL': 'https://www.amazon.com/dp/B0BVMXYB92'}, 'tentative_price': 'Unavailable', 'rating': '4.5'}


In [32]:
prompt=ChatPromptTemplate.from_messages([
    ("system", """You are a shopping assistant. Please extract the product in {user_input}. 
      use your own reasoning to provide the details of given product. Desired response will be like 
     {format_instruction}. Product price will be in USD value. If multiple products are available provide the
     best rating and most sell and best price product details.
     Product details will be like dimensions, ram size, disk size,product url etc its fully depends on product.
     if product is not available please tell that you do not have real time information , for more details asked to check
     few ecommerce sites.
    ## User Question:
    {user_input}
    ## Your Answer:
    {format_instruction}"""),
    ("human","{user_input}")])


In [33]:
chain=prompt.partial(format_instruction=details_output_parser.get_format_instructions()) | model | details_output_parser

In [ ]:
# trained LLM model output this is not real time details
result=chain.invoke({"user_input":"I want to buy a iphone 15 pro max"})
result

{'product_name': 'iPhone 15 Pro Max',
 'product_details': {'storage_options': ['256GB', '512GB', '1TB'],
  'colors': ['Space Black', 'Silver', 'Gold', 'Deep Purple'],
  'display': '6.7-inch Super Retina XDR display',
  'chip': 'A17 Pro chip',
  'camera': '48MP Main camera with advanced features',
  'ram': '6GB',
  'battery': 'All-day battery life',
  'dimensions': 'Approximately 6.28 x 3.02 x 0.32 inches',
  'weight': 'Approximately 8.28 ounces',
  'operating_system': 'iOS 17'},
 'tentative_price': '$1199',
 'rating': '4.8'}